### Leer el archivo JSON usando el "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_genre_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movies_genre_df = spark.read.schema(movies_genre_schema).json(
    f"{bronze_folder_path}/{v_file_date}/movie_genre.json"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit, concat


In [0]:
movies_genre_final_df = movies_genre_df.withColumnsRenamed({
    'movieId': 'movie_id',
    'genreId': 'genre_id'
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.movies_genres")
overwrite_partition(movies_genre_final_df, "movie_silver", "movies_genres", "file_date")


In [0]:
#movies_genre_final_df.write.mode("overwrite").partitionBy("movie_id").parquet("/mnt/datamoviehistory/silver/movies_genres")

#movies_genre_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_genres")
movies_genre_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_genres")

In [0]:
%sql
select file_date, count(1)
from movie_silver.movies_genres
group by file_date;

In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies_genres